In [83]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


# MODEL - Price - Based Filtering

In [1]:
import pandas as pd
import numpy as np
import ast
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # (playerid, library)
prices = pd.read_csv("./clean_datasets/prices.csv")  # (gameid, date_acquired, price_usd)

In [3]:
# Tạo DataFrame mở rộng từ danh sách game của mỗi người chơi
# Chỉ cần chạy 1 lần - lưu lại rồi đọc thôithôi
import os

if (not os.path.exists("./data_model/player_data_prices.csv")):
    
    player_data = []
    for _, row in purchased_games.iterrows():
        playerid = row["playerid"]
        
        games = ast.literal_eval(row["library"])  
        
        prices_list = [prices.loc[prices["gameid"] == game, "price_usd"].values[0] 
                    for game in games if game in prices["gameid"].values]

        if len(prices_list) > 0:
            player_data.append({
                "playerid": playerid,
                "mean_price": np.mean(prices_list),
                "median_price": np.median(prices_list),
                "min_price": np.min(prices_list),
                "max_price": np.max(prices_list),
                "last_5_avg": np.mean(prices_list[-5:]) if len(prices_list) >= 5 else np.mean(prices_list),
                "target_price": np.mean(prices_list[-5:])  # Nhãn dự đoán
            })

    player_data


    df1 = pd.DataFrame(player_data)
    # Write 1 lần thôi - nhớ comment lại lần sau chỉ cần đọc file
    df1.to_csv("./data_model/player_data_prices.csv", index=False)
# Hoặc đọc file 

In [4]:
# Đọc file thay vì chạy lại
df= pd.read_csv("./data_model/player_data_prices.csv")

In [5]:
# Chuẩn bị dữ liệu huấn luyện
X = df.drop(columns=["playerid", "target_price"])
y = df["target_price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình XGBoost
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

# Dự đoán giá tối ưu cho người chơi mới
def suggest_game_for_player(playerid):
    # Lấy danh sách game của người chơi từ player_data
    player_info = df[df["playerid"] == playerid]
    
    if player_info.empty:
        return "Không tìm thấy dữ liệu người chơi này."
    
    # Trích xuất thông tin về giá
    mean_price = player_info["mean_price"].values[0]
    median_price = player_info["median_price"].values[0]
    min_price = player_info["min_price"].values[0]
    max_price = player_info["max_price"].values[0]
    last_5_avg = player_info["last_5_avg"].values[0]

    # Dự đoán mức giá phù hợp
    feature_vector = np.array([[mean_price, median_price, min_price, max_price, last_5_avg]])
    predicted_price = model.predict(feature_vector)[0]

    # Tìm các game có giá gần nhất với mức giá này
    suggested_games = prices.iloc[(prices["price_usd"] - predicted_price).abs().argsort()[:10]]  # Lấy 10 game gần nhất
    
    return suggested_games



RMSE: 0.24174656623292673


In [6]:
# DDự đoán cho một người chơi mới với danh sách gameid đã mua
test_player = 76561198060698936
print(suggest_game_for_player(test_player))

        gameid date_acquired  price_usd
49635  1582650    2025-02-24  22.695552
16231   667970    2025-02-24  22.693167
67504  2116800    2025-02-24  22.710504
28225   991780    2025-02-24  22.712488
1361    200510    2025-02-24  22.679996
43721  1417880    2025-02-24  22.674023
66925  2099880    2025-02-24  22.723771
18471   724250    2025-02-24  22.723771
28452   998990    2025-02-24  22.723771
61261  1911610    2025-02-24  22.667224


# MODEL 2 - Content-Based Filtering

In [7]:
import pandas as pd
import re
import ast 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
from sklearn.decomposition import TruncatedSVD


In [9]:
# Chạy file popular game trước để lấy 10 % game phổ biến

purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # playerid, library
top_10_percent_game = pd.read_csv("./data_model/top_10_percent_games.csv")  # gameid, title, genres, description
achievements = pd.read_csv("./clean_datasets/achievements.csv")  # achievementid, gameid, title, description
history = pd.read_csv("./clean_datasets/history.csv")  # playerid, achievementid, date_acquired


KeyboardInterrupt



In [ ]:
def preprocess_text(text):
    """ Hàm tiền xử lý văn bản: chuyển thành chữ thường và loại bỏ dấu câu """
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Loại bỏ khoảng trắng thừa
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
    return text

top_10_percent_game['processed_title'] = top_10_percent_game['title'].apply(preprocess_text)
top_10_percent_game['processed_genres'] = top_10_percent_game['genres'].apply(preprocess_text)
top_10_percent_game['processed_developers'] = top_10_percent_game['developers'].apply(preprocess_text)
top_10_percent_game['processed_publishers'] = top_10_percent_game['publishers'].apply(preprocess_text)
top_10_percent_game['processed_languages'] = top_10_percent_game['supported_languages'].apply(preprocess_text)

# Tạo một cột tổng hợp các thông tin để vector hóa
top_10_percent_game['combined_features'] = top_10_percent_game['processed_title'] + ' ' + \
                              top_10_percent_game['processed_genres'] + ' ' + \
                              top_10_percent_game['processed_developers'] + ' ' + \
                              top_10_percent_game['processed_publishers'] + ' ' + \
                              top_10_percent_game['processed_languages']
top_10_percent_game

In [ ]:
import os
if (not os.path.exists('./data_model/cosine_similarity.csv')):

    # Chạy 1 lần rồi sau đó đọc file thôi
    # Tạo vector TF-IDF cho các đặc trưng tổng hợp của game
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    games_tfidf = tfidf_vectorizer.fit_transform(top_10_percent_game['combined_features'])

    # Tính toán độ tương đồng cosine giữa các game
    cosine_sim = cosine_similarity(games_tfidf)

    # Lưu kết quả cosine similarity vào CSV để tái sử dụng
    cosine_sim_df = pd.DataFrame(cosine_sim, index=top_10_percent_game['gameid'], columns=top_10_percent_game['gameid'])
    cosine_sim_df.to_csv('./data_model/cosine_similarity.csv', index=True)


In [ ]:
cosine_sim_df = pd.read_csv('./data_model/cosine_similarity.csv')
cosine_sim_df

In [ ]:
def preprocess_purchased_games(purchased_games):
    def safe_eval(val):
        if isinstance(val, str):  
            try:
                return ast.literal_eval(val)
            except (ValueError, SyntaxError):
                return []  # Trả về danh sách rỗng nếu lỗi
        elif isinstance(val, list):
            return val  
        return []  # Trả về danh sách rỗng nếu không phải list

    purchased_games['library'] = purchased_games['library'].apply(safe_eval)
    return purchased_games


In [ ]:
def recommend_games_for_player(playerid, purchased_games, cosine_sim_df, games_df, top_n=10):
    """Gợi ý game dựa trên lịch sử mua game của người chơi."""
    
    # Chuyển playerid và gameid về kiểu string để tránh lỗi
    purchased_games['playerid'] = purchased_games['playerid'].astype(str)
    playerid = str(playerid)

    # Lọc danh sách game mà người chơi đã mua
    player_games = purchased_games[purchased_games['playerid'] == playerid]

    if player_games.empty:
        print(f"⚠️ Không tìm thấy người chơi {playerid} trong danh sách purchased_games.")
        return games_df.head(top_n)[['gameid', 'title', 'genres']]  # Trả về game phổ biến
    
    # Lấy danh sách game đã mua
    purchased_game_ids = player_games['library'].values[0]
    
    # Nếu danh sách rỗng, trả về game phổ biến
    if not purchased_game_ids:
        print(f"⚠️ Người chơi {playerid} chưa mua game nào.")
        return games_df.head(top_n)[['gameid', 'title', 'genres']]
    
    # Tính điểm tương đồng
    similar_scores = pd.Series(dtype=float)
    for game_id in purchased_game_ids:
        if game_id in cosine_sim_df.index:
            similar_scores = pd.concat([similar_scores, cosine_sim_df[game_id]])
        

    # Tổng hợp điểm số, sắp xếp giảm dần
    similar_scores = similar_scores.groupby(similar_scores.index).sum().sort_values(ascending=False)
    
    # Loại bỏ game đã mua, chọn 10 game đầu tiên
    recommended_game_ids = similar_scores[~similar_scores.index.isin(purchased_game_ids)].head(top_n).index
    
    # Nếu không đủ game để gợi ý (vì không tìm thấy trong cosine_sim_df), bổ sung thêm game phổ biến
    if len(recommended_game_ids) < top_n:
        print(f"⚠️ Không đủ game gợi ý. Bổ sung thêm game phổ biến.")
        recommended_game_ids = recommended_game_ids.append(games_df[~games_df['gameid'].isin(purchased_game_ids)].head(top_n - len(recommended_game_ids))['gameid'])
    
    return games_df[games_df['gameid'].isin(recommended_game_ids)][['gameid', 'title', 'genres']]



In [ ]:
purchased_games = preprocess_purchased_games(purchased_games)  # Chuyển đổi cột 'library'

In [ ]:
playerid = 76561198060698936  
recommended_games = recommend_games_for_player(playerid, purchased_games, cosine_sim_df, top_10_percent_game)
print(recommended_games)

# MODEL 3 - Collaborative Filtering

In [10]:
%pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
import findspark

In [12]:
findspark.init()
spark = SparkSession.builder.appName("GameRecommendation").getOrCreate()

In [13]:

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import pickle


In [14]:
purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # (playerid, library)


In [15]:
purchased_games

,playerid,library
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61..."
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2..."
...,...,...
102543,76561199063275634,[]
102544,76561198003275888,"[3920, 2600, 6980, 4540, 4550, 7830, 22330, 22..."
102545,76561198944668572,[]
102546,76561198033563710,[]


In [16]:
# purchased_games['library'] = purchased_games['library'].apply(lambda x: x.split(','))


In [17]:
purchased_games

,playerid,library
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61..."
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2..."
...,...,...
102543,76561199063275634,[]
102544,76561198003275888,"[3920, 2600, 6980, 4540, 4550, 7830, 22330, 22..."
102545,76561198944668572,[]
102546,76561198033563710,[]


In [18]:
all_games = set()
for games in purchased_games["library"]:
    if isinstance(games, str):  # Nếu là chuỗi, tách game
        cleaned_games = [game.strip("[]") for game in games.split(',')]
    elif isinstance(games, list):  # Nếu là danh sách, xử lý trực tiếp
        cleaned_games = [game.strip("[]") for game in games]
    all_games.update(cleaned_games)

game_to_index = {game: idx for idx, game in enumerate(all_games)}
index_to_game = {idx: game for game, idx in game_to_index.items()}

# Tạo ánh xạ player_id → index
player_to_index = {player_id: idx for idx, player_id in enumerate(purchased_games["playerid"])}




In [19]:
all_games

{'',
 ' 659630',
 ' 798720',
 ' 504260',
 ' 435230',
 ' 628580',
 ' 557750',
 ' 1127700',
 ' 211160',
 '2062120',
 ' 2896520',
 ' 852590',
 ' 1015930',
 ' 999660',
 ' 1368820',
 ' 3298170',
 ' 2591910',
 '342180',
 ' 2963510',
 ' 1385100',
 ' 2262010',
 ' 731040',
 ' 213610',
 ' 624870',
 ' 551780',
 ' 857770',
 ' 843720',
 ' 252750',
 ' 448380',
 ' 465200',
 ' 408520',
 ' 1015720',
 ' 208710',
 ' 57970',
 ' 979710',
 '226980',
 ' 786490',
 ' 1045080',
 ' 503960',
 ' 253490',
 ' 528020',
 ' 1597550',
 ' 6400',
 ' 1725340',
 ' 508940',
 ' 320120',
 ' 2001570',
 ' 2427570',
 ' 539090',
 ' 486780',
 '375950',
 ' 359160',
 ' 2718010',
 ' 1766060',
 ' 634070',
 ' 1725640',
 ' 1216710',
 ' 383020',
 ' 1450650',
 ' 562280',
 ' 1963520',
 ' 2626240',
 ' 618310',
 ' 948740',
 ' 1173510',
 ' 1269350',
 ' 446000',
 ' 329690',
 ' 771420',
 '313740',
 ' 461860',
 ' 423760',
 ' 2017340',
 ' 1778830',
 ' 790290',
 ' 611640',
 ' 757380',
 ' 710360',
 '337950',
 ' 513650',
 ' 754330',
 ' 699040',
 ' 35

In [20]:
index_to_game

{0: '',
 1: ' 659630',
 2: ' 798720',
 3: ' 504260',
 4: ' 435230',
 5: ' 628580',
 6: ' 557750',
 7: ' 1127700',
 8: ' 211160',
 9: '2062120',
 10: ' 2896520',
 11: ' 852590',
 12: ' 1015930',
 13: ' 999660',
 14: ' 1368820',
 15: ' 3298170',
 16: ' 2591910',
 17: '342180',
 18: ' 2963510',
 19: ' 1385100',
 20: ' 2262010',
 21: ' 731040',
 22: ' 213610',
 23: ' 624870',
 24: ' 551780',
 25: ' 857770',
 26: ' 843720',
 27: ' 252750',
 28: ' 448380',
 29: ' 465200',
 30: ' 408520',
 31: ' 1015720',
 32: ' 208710',
 33: ' 57970',
 34: ' 979710',
 35: '226980',
 36: ' 786490',
 37: ' 1045080',
 38: ' 503960',
 39: ' 253490',
 40: ' 528020',
 41: ' 1597550',
 42: ' 6400',
 43: ' 1725340',
 44: ' 508940',
 45: ' 320120',
 46: ' 2001570',
 47: ' 2427570',
 48: ' 539090',
 49: ' 486780',
 50: '375950',
 51: ' 359160',
 52: ' 2718010',
 53: ' 1766060',
 54: ' 634070',
 55: ' 1725640',
 56: ' 1216710',
 57: ' 383020',
 58: ' 1450650',
 59: ' 562280',
 60: ' 1963520',
 61: ' 2626240',
 62: ' 61

In [21]:
player_to_index

{76561198060698936: 0,
 76561198287452552: 1,
 76561198040436563: 2,
 76561198042412488: 3,
 76561198119605821: 4,
 76561198049686270: 5,
 76561198155814250: 6,
 76561198083492916: 7,
 76561198150634683: 8,
 76561198836367256: 9,
 76561198131958442: 10,
 76561198106910534: 11,
 76561198122070915: 12,
 76561198740883087: 13,
 76561198016756834: 14,
 76561198072740562: 15,
 76561198144540723: 16,
 76561198096557299: 17,
 76561198884939938: 18,
 76561198220441373: 19,
 76561198046610425: 20,
 76561198801797946: 21,
 76561199000909663: 22,
 76561199004958958: 23,
 76561198120109043: 24,
 76561198856855325: 25,
 76561198146275005: 26,
 76561197989446733: 27,
 76561197997318138: 28,
 76561198015133782: 29,
 76561199158139117: 30,
 76561197962051588: 31,
 76561198191613675: 32,
 76561198080851200: 33,
 76561198230516004: 34,
 76561198282337972: 35,
 76561198391791168: 36,
 76561198213729077: 37,
 76561199248076925: 38,
 76561198234203304: 39,
 76561199225844826: 40,
 76561198087768441: 41,
 7

In [22]:
game_to_index

{'': 0,
 ' 659630': 1,
 ' 798720': 2,
 ' 504260': 3,
 ' 435230': 4,
 ' 628580': 5,
 ' 557750': 6,
 ' 1127700': 7,
 ' 211160': 8,
 '2062120': 9,
 ' 2896520': 10,
 ' 852590': 11,
 ' 1015930': 12,
 ' 999660': 13,
 ' 1368820': 14,
 ' 3298170': 15,
 ' 2591910': 16,
 '342180': 17,
 ' 2963510': 18,
 ' 1385100': 19,
 ' 2262010': 20,
 ' 731040': 21,
 ' 213610': 22,
 ' 624870': 23,
 ' 551780': 24,
 ' 857770': 25,
 ' 843720': 26,
 ' 252750': 27,
 ' 448380': 28,
 ' 465200': 29,
 ' 408520': 30,
 ' 1015720': 31,
 ' 208710': 32,
 ' 57970': 33,
 ' 979710': 34,
 '226980': 35,
 ' 786490': 36,
 ' 1045080': 37,
 ' 503960': 38,
 ' 253490': 39,
 ' 528020': 40,
 ' 1597550': 41,
 ' 6400': 42,
 ' 1725340': 43,
 ' 508940': 44,
 ' 320120': 45,
 ' 2001570': 46,
 ' 2427570': 47,
 ' 539090': 48,
 ' 486780': 49,
 '375950': 50,
 ' 359160': 51,
 ' 2718010': 52,
 ' 1766060': 53,
 ' 634070': 54,
 ' 1725640': 55,
 ' 1216710': 56,
 ' 383020': 57,
 ' 1450650': 58,
 ' 562280': 59,
 ' 1963520': 60,
 ' 2626240': 61,
 ' 618310

In [23]:
import os
if (not os.path.exists("./data_model/interaction_matrix.npz")):

    from scipy.sparse import lil_matrix, save_npz, load_npz

    num_players = len(purchased_games)
    num_games = len(all_games)
    interaction_matrix = lil_matrix((num_players, num_games), dtype=np.uint8)

    for i, games in enumerate(purchased_games["library"]):
        if isinstance(games, str):  # Nếu games là chuỗi, chuyển nó thành danh sách
            games = games.strip("[]").split(",")  # Tách chuỗi thành danh sách

        for game in games:
            game = str(game).strip()  # Đảm bảo game ID là chuỗi không có khoảng trắng

            if game in game_to_index:  # Kiểm tra game có tồn tại trong game_to_index không
                # if i < 1:     
                    # print("Abc")
                    # print(game_to_index[game])

                interaction_matrix[i, game_to_index[game]] = 1

    # Chuyển ma trận sang CSR (tối ưu tính toán)
    interaction_matrix = interaction_matrix.tocsr()

    # Lưu ma trận vào file để tái sử dụng mà không cần tính toán lại
    save_npz("./data_model/interaction_matrix.npz", interaction_matrix)




In [24]:
from scipy.sparse import lil_matrix, save_npz, load_npz

interaction_matrix = load_npz("./data_model/interaction_matrix.npz")


In [25]:
data = []
for player_id, games in enumerate(interaction_matrix):
    for game_index in games.nonzero()[1]:
        data.append((int(player_id), int(game_index), float(1)))  # Chuyển đổi kiểu dữ liệu

In [26]:
interaction_matrix

<102548x45093 sparse matrix of type '<class 'numpy.uint8'>'
	with 6139343 stored elements in Compressed Sparse Row format>

In [27]:
data

[(0, 70, 1.0),
 (0, 91, 1.0),
 (0, 117, 1.0),
 (0, 128, 1.0),
 (0, 137, 1.0),
 (0, 150, 1.0),
 (0, 163, 1.0),
 (0, 199, 1.0),
 (0, 209, 1.0),
 (0, 211, 1.0),
 (0, 262, 1.0),
 (0, 377, 1.0),
 (0, 398, 1.0),
 (0, 409, 1.0),
 (0, 546, 1.0),
 (0, 603, 1.0),
 (0, 633, 1.0),
 (0, 656, 1.0),
 (0, 664, 1.0),
 (0, 677, 1.0),
 (0, 745, 1.0),
 (0, 807, 1.0),
 (0, 808, 1.0),
 (0, 901, 1.0),
 (0, 931, 1.0),
 (0, 1004, 1.0),
 (0, 1010, 1.0),
 (0, 1034, 1.0),
 (0, 1087, 1.0),
 (0, 1102, 1.0),
 (0, 1131, 1.0),
 (0, 1154, 1.0),
 (0, 1158, 1.0),
 (0, 1170, 1.0),
 (0, 1184, 1.0),
 (0, 1188, 1.0),
 (0, 1201, 1.0),
 (0, 1211, 1.0),
 (0, 1217, 1.0),
 (0, 1276, 1.0),
 (0, 1292, 1.0),
 (0, 1304, 1.0),
 (0, 1329, 1.0),
 (0, 1343, 1.0),
 (0, 1357, 1.0),
 (0, 1398, 1.0),
 (0, 1453, 1.0),
 (0, 1508, 1.0),
 (0, 1541, 1.0),
 (0, 1630, 1.0),
 (0, 1632, 1.0),
 (0, 1666, 1.0),
 (0, 1752, 1.0),
 (0, 1836, 1.0),
 (0, 1862, 1.0),
 (0, 2007, 1.0),
 (0, 2027, 1.0),
 (0, 2060, 1.0),
 (0, 2090, 1.0),
 (0, 2155, 1.0),
 (0, 21

In [28]:
# Tạo DataFrame PySpark
columns = ["player_id", "game_id", "rating"]
df = spark.createDataFrame(data, columns)

In [29]:
df = df.withColumn("player_id", col("player_id").cast("int"))
df = df.withColumn("game_id", col("game_id").cast("int"))
df = df.withColumn("rating", col("rating").cast("float"))
df.printSchema()  # Xác nhận lại kiểu dữ liệu


root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- rating: float (nullable = true)



In [30]:
df

DataFrame[player_id: int, game_id: int, rating: float]

In [31]:
print(spark)


In [32]:
df.printSchema()


root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- rating: float (nullable = true)



In [33]:
from pyspark.sql.functions import col, count

In [34]:
# df.count() # Kiểm tra số dòng trong DataFrame
# print(df.schema)  # Xem kiểu dữ liệu của từng cột


In [35]:
# df.describe().show()  # Kiểm tra thống kê dữ liệu
# df.select([col(c).isNull().sum() for c in df.columns]).show()  # Kiểm tra giá trị null
# df.show(5)  # Xem thử 5 dòng đầu

In [36]:
# Khởi tạo mô hình ALS
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=10,         # Số vòng lặp
    regParam=0.1,       # Hệ số điều chuẩn
    userCol="player_id",
    itemCol="game_id",
    ratingCol="rating",
    coldStartStrategy="drop"  # Loại bỏ dữ liệu không đủ thông tin
)

# Huấn luyện mô hình
model = als.fit(df)


In [37]:
interaction_matrix

<102548x45093 sparse matrix of type '<class 'numpy.uint8'>'
	with 6139343 stored elements in Compressed Sparse Row format>

In [38]:
interaction_matrix_dense = interaction_matrix.toarray()


In [43]:
interaction_matrix_dense

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [44]:

from scipy.sparse.linalg import svds
import numpy as np
import pickle

# Convert the existing interaction_matrix to float type
interaction_matrix = interaction_matrix.astype(float)

# Now perform SVD
num_components = 50
U_svd, sigma_svd, Vt_svd = svds(interaction_matrix, k=num_components)

# Convert sigma to diagonal matrix
sigma_svd = np.diag(sigma_svd)

# Save results for reuse
with open("./data_model/svd_model.pkl", "wb") as f:
    pickle.dump((U_svd, sigma_svd, Vt_svd), f)

In [45]:
# density = interaction_matrix.nnz / (interaction_matrix.shape[0] * interaction_matrix.shape[1])
# print(f"Mật độ dữ liệu không rỗng: {density:.6f}")

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similar_users(player_index, user_embeddings_sparse, top_k=20):
    """
    Lấy danh sách top_k người chơi có độ tương đồng cao nhất với player_index
    """
    similarities = cosine_similarity(user_embeddings_sparse[player_index], user_embeddings_sparse).flatten()
    # print(similarities)
    similar_users = np.argsort(similarities)[::-1][1:top_k + 1]  # Lấy top_k người chơi gần nhất
    return similar_users, similarities

In [53]:
def recommend_games(player_id, U_svd, sigma_svd, Vt_svd, interaction_matrix, 
                    game_to_index, index_to_game, player_to_index, num_recommendations=10):
    
    # Kiểm tra xem player_id có trong player_to_index không
    if player_id not in player_to_index:
        print("❌ Player ID không hợp lệ.")
        return []

    # Chuyển player_id thành index
    player_index = player_to_index[player_id]

    # Tạo sparse embedding để tiết kiệm bộ nhớ
    from scipy.sparse import csr_matrix
    user_embeddings_sparse = csr_matrix(U_svd @ sigma_svd)
    print(user_embeddings_sparse)
    # Lấy danh sách người chơi tương tự
    similar_users, similarities = get_similar_users(player_index, user_embeddings_sparse)
    print(similar_users)
    print(similarities[similar_users])
    # Tập hợp các game mà những người chơi tương tự đã mua
    recommended_games = set()
    for similar_user in similar_users:
        recommended_games.update(interaction_matrix[similar_user].nonzero()[1])
    print(recommended_games)
    # Loại bỏ game mà người chơi hiện tại đã mua
    purchased_games = set(interaction_matrix[player_index].nonzero()[1])
    final_recommendations = [index_to_game[idx] for idx in recommended_games if idx not in purchased_games]

    return final_recommendations[:num_recommendations]  # Lấy đúng số lượng cần thiết


In [54]:
player_id = 76561198060698936  # ID của người chơi cần gợi ý
recommended_games = recommend_games(player_id, U_svd, sigma_svd, Vt_svd, 
                                    interaction_matrix, game_to_index, index_to_game, player_to_index)

print("🎮 Game được đề xuất:", recommended_games)


  (0, 0)	-0.3206284879160034
  (0, 1)	0.1796199228360866
  (0, 2)	-0.5406230033295795
  (0, 3)	-1.57296513943077
  (0, 4)	-0.958345607962977
  (0, 5)	-1.4299063662636753
  (0, 6)	-1.0183641458023693
  (0, 7)	3.448572289910201
  (0, 8)	1.3744843397172848
  (0, 9)	-0.9452694785642132
  (0, 10)	-0.3401382960152719
  (0, 11)	0.8496160154653097
  (0, 12)	-0.4643019523267732
  (0, 13)	-0.3584271840268045
  (0, 14)	-0.3909836177125208
  (0, 15)	-1.8755136717119878
  (0, 16)	0.2578508332868222
  (0, 17)	1.35497622722054
  (0, 18)	3.6045352908915067
  (0, 19)	-0.028755695441378456
  (0, 20)	-3.606245935751887
  (0, 21)	-0.648674813266542
  (0, 22)	-0.8555720446086378
  (0, 23)	-1.1127472117431645
  (0, 24)	-0.7424559011773172
  :	:
  (102544, 25)	-1.1856069977886905
  (102544, 26)	0.2829088872232186
  (102544, 27)	0.8512419501899797
  (102544, 28)	-0.6726234331584129
  (102544, 29)	-0.1473931887358496
  (102544, 30)	-0.1767439971313808
  (102544, 31)	1.203320164941232
  (102544, 32)	0.119999852